## Final Project Submission

Please fill out:
* __Student name__: Hannah Kwirikia
* __Student pace__: full time
* __Scheduled project review date/time__: 20th Nov.2022/ 11:59PM
* __Instructor name__: Mark Tiba 
* __Blog post URL__:


# Microsoft's Movie Market Analysis - Exploratory Data Analysis

## Introduction

## Research Objectives

- To find out what types of films are currently doing the best at the box office.

- To find out factors affecting film performance

## Analysis Plan

Here are the steps I will follow for my research:
    
1. Import data
2. Unzip data
3. Merge datasets
4. Data cleaning
4. Data visualization

## Research Questions

1. What films have highest domestic gross?
2. What are the top studios by no. of movies?
3. Does length affect the grossing?
4. Which movies have the highest rating?

## Data Description

This project uses two data files namely:
- __im.db.zip__ extracted from __Box Office__
- __boom.movie_gross.csv.gz__ extracted from __IMDB__



In [ ]:
cd C:\\Users\\Win 10 Pro\\Desktop\\dsc-phase-1-project-v2-4\\zippedData

## Import Libraries

In [ ]:
#importing libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
%matplotlib inline

### Import and Unzip First Dataset (im.db.zip)

In [ ]:
#to unzip IMDB file
import zipfile
with zipfile.ZipFile('im.db.zip', 'r') as my_zip:
    my_zip.extractall('files') # the unzipped files are now in a folder named files

In [ ]:
cd C:\\Users\\Win 10 Pro\\Desktop\\dsc-phase-1-project-v2-4\\zippedData\\files

In [ ]:
#to view list of tables
conn = sqlite3.connect('im.db')
cur = conn.cursor()
cur.execute("""SELECT name FROM sqlite_master WHERE type = 'table';""")
table_names = cur.fetchall()
table_names

The tables for the 'movie_basics' and 'movie_ratings' are the most important. We will look at them below.

#### movie_basics table

In [ ]:
#to view the movie_basics table
movie_basics = pd.read_sql("""SELECT * FROM movie_basics;""", conn )
movie_basics

#### movie_ratings table

In [ ]:
#to view the movie_ratings table
movie_ratings = pd.read_sql("""SELECT * FROM movie_ratings;""", conn )
movie_ratings

#### Merging movie_basics and movie_ratings table

In [ ]:
#merge movie_basics and movie_ratings tables
merged_movies = pd.merge(left=movie_basics, right=movie_ratings, on='movie_id')
merged_movies.head(10)

### Import and Unzip Second Dataset(boom.movie_gross.csv.gz)

In [ ]:
cd C:\\Users\\Win 10 Pro\\Desktop\\dsc-phase-1-project-v2-4\\zippedData

In [ ]:
#unzip .gz file
import gzip
with gzip.open('bom.movie_gross.csv.gz') as f:

    bom_df = pd.read_csv(f)

#### Preview Merged Dataset

In [ ]:
#preview the data 
bom_df.head()

In [ ]:
#rename title column to primary_title to enable merging
bom_df.rename(columns = {'title':'primary_title'}, inplace = True)
bom_df.head()

### Merge First Data with Second Data

In [ ]:
#merge first data with second data
df = pd.merge(left = merged_movies, right = bom_df, on = 'primary_title')
df.head()

In [ ]:
#learn more about the data
df.shape

The merged data has 3027 rows and 12 columns

In [ ]:
#summary of the data overview
df.info()

## Data Cleaning

The data cleaning process will entail three main steps:
- looking for duplicates
- looking for missing values
- check outliers

### Look for any duplicates

Below I defined a function to detect duplicates.

In [ ]:
# Duplicated entries
def identify_duplicates(data):
    """A function to identify any duplicates"""
    # identify the duplicates 
    # empty list to store Bool results from duplicated
    duplicates = []
    for i in data.duplicated():
        duplicates.append(i)
    # identify if there is any duplicates. 
    duplicates_set = set(duplicates) 
    if (len(duplicates_set) == 1):
        print("The Data has no duplicates")
    else:
        no_true = 0
        for val in duplicates:
            if (val == True):
                no_true += 1
        # percentage of the data represented by duplicates 
        duplicates_percentage = np.round(((no_true / len(data)) * 100), 3)
        print(f"The Data has {no_true} duplicated rows.\nThis constitutes {duplicates_percentage}% of the data set.") 



identify_duplicates(df)

#### Check for Duplicates in the Primary Key column
Columns that have unique details such as the movie_id should not contain any duplicates

In [ ]:
def unique_column_duplicates(data, column):
    """handling duplicates in unique column"""
    # empty list to store the duplicate bools
    duplicates = []
    for i in data[column].duplicated():
        duplicates.append(i)
    
    # identify if there are any duplicates
    duplicates_set = set(duplicates)
    if (len(duplicates_set) == 1):
        print(f"The column {column.title()} has no duplicates")
    else:
        no_true = 0
        for val in duplicates:
            if (val == True):
                no_true += 1
        # percentage of the data represented by duplicates 
        duplicates_percentage = np.round(((no_true / len(data)) * 100), 3)
        print(f"The column {column.title()} has {no_true} duplicated rows.\nThis constitutes {duplicates_percentage}% of the data set.")


unique_column_duplicates(df, "movie_id")

Because the movie_id are our unique identifiers,  are unique. Thus we need to remove any duplicates

In [ ]:
# handling the unique column duplicates 
def remove_unique_column_duplicates(data, column):
    """Simple Function to remove duplicated rows"""
    data.drop_duplicates(subset=column, keep="first", inplace=True)
    # confirm if the duplicated rows have been removed
    confirm = unique_column_duplicates(df, "movie_id")

    return confirm 


remove_unique_column_duplicates(df, "movie_id")

The duplicates along the movie_id column have now been removed.

### Percentage of missing values per column

In [ ]:
# identify missing 
def identify_missing_values(data):
    """Identify is the data has missing values"""
    # identify if data has missing values(data.isnull().any())
    # empty dict to store missing values
    missing = []
    for i in data.isnull().any():
        # add the bool values to empty list 
        missing.append(i)
    # covert list to set (if data has missing value, the list should have true and false)
    missing_set = set(missing)
    if (len(missing_set) == 1):
        out = print("The Data has no missing values")
    else:
        out = print("The Data has missing values.")

    return out


identify_missing_values(df)

In [ ]:
#identify missing values
def missing_values(data):
    """A simple function to identify data has missing values"""
    # identify the total missing values per column
    # sort in order 
    miss = data.isnull().sum().sort_values(ascending = False)

    # calculate percentage of the missing values
    percentage_miss = (data.isnull().sum() / len(data)).sort_values(ascending = False)

    # store in a dataframe 
    missing = pd.DataFrame({"Missing Values": miss, "Percentage(%)": percentage_miss})

    # remove values that are missing 
    missing.drop(missing[missing["Percentage(%)"] == 0].index, inplace = True)

    return missing


missing_data = missing_values(df)
missing_data

In [ ]:
def graph_missing(data):
    """Graphically represent the missing values"""
    fig, axes = plt.subplots(figsize = (6, 3))

    sns.barplot(x = data.index , y = data["Missing Values"]) 
    plt.xlabel("Columns", fontdict={"color": "black",  "size": 15}) 
    plt.xticks(rotation = '60') 
    plt.ylabel("Missing Values", fontdict={"color": "black",  "size": 15}) 
    plt.title("Missing Values per Column", fontsize = 18)
    plt.show()

    # save the plot 
    fig.savefig("missing.png")


graph_missing(missing_data)

Since there are only 3 rows missing in the studio, we can drop those.

In [ ]:
df = df.dropna(subset=['studio'])

Since there are only 7 row missing in the genre, we can drop that.

In [ ]:
df = df.dropna(subset=['genres'])

The domestic_gross column has less than 1% (22 values) missing values. Thus, we will replace the missing value by the mean domestic_gross income.

In [ ]:
#replace missing values in domestic_gross with mean
df['domestic_gross'] = df['domestic_gross'].fillna(df['domestic_gross'].mean())

The runtime_minutes column also has less than 1%(47 values) of missing values. Thus, we will replace the missing value by the mean runtime_minutes.

In [ ]:
#replace missing values in runtime_minutes with mean
df['runtime_minutes'] = df['runtime_minutes'].fillna(df['runtime_minutes'].mean())

The foreign_gross column  has the highest number of missing values at about 40%. Thus, we drop that column.

In [ ]:
df.drop(columns = 'foreign_gross', inplace = True)

### Check for Outliers

#### Define a Funtion to Detect Outliers

In [ ]:
def IQR_outliers(data, column):
    """Simple function to identify and remove outliers using IQR"""
    # get Q1 and Q2
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    # calculate IQR
    IQR = Q3-Q1
    # identify ouliers (returns True if an outlier)
    out = ((data[column]<(Q1-1.5*IQR)) | (data[column]>(Q3+1.5*IQR)))

    # identify the outlier using index
    outliers_list = list(out[out==True].index)
    # identify the actual outliers using index
    outliers = []
    for val in outliers_list:
        outliers.append(data[column][val])

    
    return outliers, Q1, Q3, IQR


##### 1. domestic_gross

In [ ]:
domestic_gross_outliers = IQR_outliers(df, "domestic_gross")
domestic_gross_outliers[0]

In [ ]:
def IQR_outliers(data, column):
    """Simple function to identify and remove outliers using IQR"""
    # get Q1 and Q2
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    # calculate IQR
    IQR = Q3-Q1
    # identify ouliers (returns True if an outlier)
    out = ((data[column]<(Q1-1.5*IQR)) | (data[column]>(Q3+1.5*IQR)))

    # identify the outlier using index
    outliers_list = list(out[out==True].index)
    # identify the actual outliers using index
    outliers = []
    for val in outliers_list:
        outliers.append(data[column][val])

    
    return outliers, Q1, Q3, IQR


domestic_gross_outliers = IQR_outliers(df, "domestic_gross")
domestic_gross_outliers[0]

The domestic_gross_outliers are possible values thus we will leave them as they are. 

##### 2. start year

The start year has no outliers as shown below

In [ ]:
start_year_outliers = IQR_outliers(df, "start_year")
start_year_outliers[0]

In [ ]:
# use a box plot to identify outliers

sns.boxplot(data=df, x="start_year")

##### 3. Runtime minutes

In [ ]:
runtime_minutes_outliers = IQR_outliers(df, "runtime_minutes")
runtime_minutes_outliers[0]

The runtime_minutes_outliers are possible values thus we will leave them as they are.

##### 4. averagerating

In [ ]:
averagerating_outliers = IQR_outliers(df, "averagerating")
averagerating_outliers[0]

The averagerating_outliers are possible values thus we will leave them as they are. This is further backed by the box plot above.

##### 5. numvotes

In [ ]:
numvotes_outliers = IQR_outliers(df, "numvotes")
numvotes_outliers[0]

The numvotes_outliers are possible values thus we will leave them as they are. 

##### 6. year

In [ ]:
year_outliers = IQR_outliers(df, "year")
year_outliers[0]

In [ ]:
# use a box plot to identify outliers

sns.boxplot(data=df, x="year")

The start year has no outliers as shown above

### Final Data

In [ ]:
#preview first 5 rows
df.head()

In [ ]:
#checking for missing values
df.isna().sum()

# Exploratory Analysis and visualization

### RQ1: What Movie Genres Have the Highest Domestic Gross?

In [ ]:
#sorting to find movies with highest domestic_gross per genre
gross_by_genre = df.sort_values('domestic_gross',ascending = False).reset_index(drop=True)

In [ ]:
gross_by_genre.loc[:, ["primary_title", "genres", "domestic_gross", "studio"]].head(10)

In [218]:
x = gross_by_genre['genres'][:20].values

In [219]:
y = gross_by_genre['domestic_gross'][:20]

In [ ]:
sns.barplot(x = gross_by_genre['genres'].values, y = gross_by_genre['domestic_gross'][:20], data = gross_by_genre)

In [ ]:
sns.barplot(x = gross_by_genre['genres'].values, y = gross_by_genre['domestic_gross'][:20], hue = gross_by_genre['studio'][:20], data =gross_by_genre)

In [ ]:
# plotting the data above

fig, ax= plt.subplots(figsize=(20,12))

ax1 = sns.barplot(data = gross_by_genre, x = 'genres', y = 'domestic_gross', ax = ax1, dodge=False)

labels = list(gross_by_genere['genres']
ax1.set_title('Genre vs Domestic Gross', fontsize=20)
ax1.set_xlabel("Movie Genres",fontsize=18)
ax1.set_ylabel("Domestic Gross", fontsize=18)
ax1.legend(title='Studio', fontsize=18, loc=1)
ax1.set_ylim(500)
ax1.set_xlim(20)
ax1.set_xticklabels(labels = x, rotation = 60,fontsize=14)
ax1.set_yticklabels(labels = y,fontsize=14)

ax1.set_xticklabels(labels = x, rotation = 90)
ax1.set_ylim(6.5, 8.5)
fig.savefig('Genre vs Domestic - Gross.png')

### RQ2: Do Highly Rated Movies Earn High Domestic Gross?

In [ ]:
#sorting to find movies with highest domestic_gross per genre
highly_rated_gross = gross_by_genre.sort_values('averagerating', ascending = False).reset_index(drop=True)
highly_rated_gross.loc[:, ["primary_title","averagerating", "genres", "domestic_gross"]].head(10)

### RQ#: Do longer movies translate to higher income?

## Ask and Answer Question

Is domestic gross capital or foreign gross higher
does year affect gross and foreign gross
studio vs gross income
What are the top studios by no. of titles?
WHat are the top studios by gross income? Domestic and foreign?
most studios have very few titles

most movies are around 100 minutes long with a average of 108 minutes,
does the distribution of depended on movie type